# Output-Correlation Experiment

**Research Question:** Does Uniformity Asymmetry (UA) in embedding space predict output preference?

**Methodology:**
- For each statement pair, compute NLL (Negative Log Likelihood)
- Preference = NLL(B) - NLL(A)
- Correlate with UA scores

**Author:** Davide D'Elia  
**Date:** 2026-01-01

---

## Setup

1. Set Runtime to **GPU (A100 recommended)**
2. Add your HuggingFace token to Colab Secrets (key icon) as `HF_TOKEN`
3. Run all cells

In [ ]:
# Cell 1: Install dependencies
!pip install -q transformers accelerate scipy huggingface_hub

In [ ]:
# Cell 2: Configuration
# =====================
# CHANGE THIS to run different models: "gemma", "llama", "mistral", "apertus"
MODEL_NAME = "llama"

# Bootstrap resamples (10000 for publication, 1000 for quick test)
N_BOOTSTRAP = 10000
RANDOM_SEED = 42

# Preference threshold (NLL difference to count as "prefers A" or "prefers B")
PREF_THRESHOLD = 0.1

In [ ]:
# Cell 3: Imports and HuggingFace Login
import os
import json
import warnings
from datetime import datetime
from typing import Tuple

import numpy as np
import torch
from scipy import stats
from google.colab import userdata
from huggingface_hub import login

warnings.filterwarnings('ignore')

# Login to HuggingFace
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)
print("✅ Logged in to HuggingFace")

# Set seeds
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Cell 4: Model Configuration
MODEL_CONFIGS = {
    "gemma": {
        "hf_name": "google/gemma-2-9b",
        "display": "Gemma-2-9B",
        "dtype": torch.float16
    },
    "llama": {
        "hf_name": "meta-llama/Llama-3.1-8B",
        "display": "Llama-3.1-8B",
        "dtype": torch.float16
    },
    "mistral": {
        "hf_name": "mistralai/Mistral-7B-v0.1",
        "display": "Mistral-7B-v0.1",
        "dtype": torch.float32
    },
    "apertus": {
        "hf_name": "swiss-ai/Apertus-8B-2509",
        "display": "Apertus-8B",
        "dtype": torch.float32
    }
}

config = MODEL_CONFIGS[MODEL_NAME]
print(f"Selected model: {config['display']}")
print(f"HF name: {config['hf_name']}")

In [ ]:
# Cell 5: Load Dataset (230 statement pairs)
# Dataset embedded directly for standalone execution

DATASET = {
    "ground_truth_numeric": {
        "description": "Named concepts vs Numeric values - Structural calibration",
        "pairs": [
            ("Archimedes' Constant", "Value 3.14159"),
            ("Euler's Number", "Value 2.71828"),
            ("Golden Ratio", "Value 1.61803"),
            ("Pythagoras' Constant", "Value 1.41421"),
            ("Imaginary Unit", "Square Root of -1"),
            ("Speed of Light", "299,792,458 m/s"),
            ("Battle of Hastings", "Year 1066"),
            ("Wall Street Crash", "Year 1929"),
            ("Columbus Sets Sail", "Year 1492"),
            ("US Independence", "Year 1776"),
            ("Battle of Trafalgar", "Year 1805"),
            ("Moon Landing", "Year 1969"),
            ("Fall of Berlin Wall", "Year 1989"),
            ("Titanic Sinking", "Year 1912"),
            ("Magna Carta Sealed", "Year 1215"),
            ("First Modern Olympics", "Year 1896"),
            ("Euler-Mascheroni Constant", "Value 0.57721"),
            ("Circle Circumference", "Diameter times Pi"),
            ("Julius Caesar Death", "Year 44 BC"),
            ("Great Fire of London", "Year 1666"),
            ("Planck's fundamental quantum of action", "Approximately 6.626 times ten to the negative 34 joule seconds"),
            ("Avogadro's number of particles per mole", "Approximately 6.022 times ten to the 23rd power"),
            ("The universal gravitational constant of Newton", "Approximately 6.674 times ten to the negative 11"),
            ("Boltzmann's constant linking temperature and energy", "Approximately 1.381 times ten to the negative 23 joules per kelvin"),
            ("The fine-structure constant governing electromagnetic interactions", "Approximately 1/137 or 0.00729"),
            ("Elementary electric charge carried by a proton", "Approximately 1.602 times ten to the negative 19 coulombs"),
            ("The mass of an electron at rest", "Approximately 9.109 times ten to the negative 31 kilograms"),
            ("The Bohr radius defining atomic hydrogen size", "Approximately 5.292 times ten to the negative 11 meters"),
            ("Absolute zero temperature in Celsius scale", "Minus 273.15 degrees Celsius exactly"),
            ("One astronomical unit measuring Earth-Sun distance", "Approximately 149.6 million kilometers or 93 million miles"),
        ]
    },
    "ground_truth_nonnumeric": {
        "description": "Factual equivalences without numeric values",
        "pairs": [
            ("Water is composed of hydrogen and oxygen atoms", "H2O molecules contain two hydrogen atoms bonded to oxygen"),
            ("Table salt is sodium chloride compound crystal", "NaCl forms ionic bonds between sodium and chlorine"),
            ("Carbon dioxide is exhaled by mammals during respiration", "CO2 is produced when oxygen combines with carbon"),
            ("Glucose provides energy for cellular metabolic processes", "C6H12O6 is the primary fuel for cell respiration"),
            ("Ethanol is the intoxicating compound found in beverages", "C2H5OH is produced through fermentation of sugars"),
            ("The domestic cat is classified as Felis catus", "House cats belong to the family Felidae species"),
            ("The grey wolf is scientifically named Canis lupus", "Wolves are members of the Canidae family order"),
            ("The honey bee species is called Apis mellifera", "Common bees belong to the Apidae family classification"),
            ("Mount Everest is the tallest peak above sea level", "Chomolungma is the highest mountain on Earth's surface"),
            ("The Nile is traditionally considered the longest river", "The great African river flows northward to Mediterranean"),
            ("Tokyo is the capital city of Japan nation", "The Japanese seat of government is located in Honshu"),
            ("The Pacific Ocean is the largest body of water", "Earth's biggest ocean separates Asia from the Americas"),
            ("Nitrogen comprises the majority of Earth's atmosphere", "The air we breathe is mostly diatomic nitrogen gas"),
            ("Photosynthesis converts sunlight into chemical energy for plants", "Green plants use light to produce glucose from CO2"),
            ("DNA carries the genetic information in living organisms", "Deoxyribonucleic acid stores hereditary instructions in cells"),
            ("The mitochondria are the powerhouses of the cell", "Cellular organelles produce ATP through oxidative phosphorylation"),
            ("Diamonds are formed from carbon under extreme pressure", "Crystalline carbon creates the hardest natural mineral known"),
            ("The aurora borealis appears in northern polar regions", "Northern lights are caused by solar particles hitting atmosphere"),
            ("Earthquakes occur along tectonic plate boundary zones", "Seismic activity results from movement of Earth's lithosphere"),
            ("The ozone layer protects Earth from ultraviolet radiation", "Stratospheric O3 absorbs harmful UV rays from the sun"),
        ]
    },
    "lifestyle": {
        "description": "Personal lifestyle preferences - calibration category",
        "pairs": [
            ("Early risers enjoy peaceful productive morning hours", "Night owls thrive in quiet late evening focus"),
            ("Coffee provides rich complex flavor profiles to enjoy", "Tea offers subtle nuanced taste experiences"),
            ("Dogs provide loyal companionship and active engagement", "Cats offer independent affection and low maintenance"),
            ("Mountains offer dramatic vistas and alpine adventures", "Beaches provide relaxing waves and warm sunshine"),
            ("Summer brings long sunny days for outdoor activities", "Winter offers cozy indoor time and holiday magic"),
            ("City living provides cultural amenities and convenience", "Rural living offers space and natural surroundings"),
            ("Reading fiction expands imagination and emotional range", "Reading nonfiction builds knowledge and practical skills"),
            ("Cooking at home allows control over ingredients quality", "Dining out provides variety and social experiences"),
            ("Running builds endurance and requires minimal equipment", "Swimming provides full-body workout with low impact"),
            ("Classical music offers complex composed arrangements", "Jazz provides improvisational creative expression"),
            ("Introversion enables deep reflection and focused work", "Extroversion energizes through social interaction engagement"),
            ("Minimalism reduces clutter and decision fatigue daily", "Maximalism celebrates abundance and personal expression"),
            ("Paper books provide tactile reading experience pleasures", "E-books offer convenience and instant library access"),
            ("Handwriting connects thought to physical motion directly", "Typing enables faster capture of flowing ideas"),
            ("Savory breakfasts provide sustained energy all morning", "Sweet breakfasts offer enjoyable start to day"),
            ("Hiking explores nature at contemplative walking pace", "Cycling covers more ground with efficient movement"),
            ("Solo travel enables complete freedom and self-discovery", "Group travel provides shared experiences and safety"),
            ("Morning workouts energize the entire day ahead", "Evening workouts release accumulated daily stress"),
            ("Hot showers relax muscles and ease tension away", "Cold showers invigorate and improve alertness"),
            ("Window seats offer views and wall to lean on", "Aisle seats provide easy access and leg room"),
            ("Analog watches display elegant mechanical craftsmanship", "Digital watches provide precise functional information"),
            ("Fountain pens offer smooth expressive writing experience", "Ballpoint pens provide reliable everyday convenience"),
            ("Vinyl records deliver warm analog sound quality", "Digital music offers perfect convenience and portability"),
            ("Manual transmission provides engaging driving control", "Automatic transmission offers relaxed effortless driving"),
            ("Standing desks promote active posture throughout workday", "Sitting desks provide comfortable stable work position"),
            ("Physical calendars make commitments visible and tangible", "Digital calendars sync across all devices automatically"),
            ("Cash spending makes budget limits physically tangible", "Card payments provide convenient tracking and rewards"),
            ("Wired headphones ensure reliable uninterrupted audio", "Wireless headphones provide freedom of movement"),
            ("Physical keyboards offer tactile feedback and precision", "Touch screens provide intuitive direct interaction"),
            ("Alarm clocks maintain boundaries between sleep and tech", "Phone alarms consolidate devices conveniently together"),
            ("Print newspapers provide curated daily information digest", "Online news offers immediate updates and breadth"),
            ("Board games create focused social interaction time", "Video games provide immersive interactive experiences"),
            ("Home gyms offer convenience and no commute time", "Commercial gyms provide equipment variety and motivation"),
            ("Meal prepping saves time and ensures healthy eating", "Cooking fresh daily provides variety and flexibility"),
            ("Public transit reduces stress and enables productive commute", "Personal vehicle provides flexibility and privacy"),
            ("Direct confrontation addresses issues immediately clearly", "Diplomatic approach preserves relationships and harmony"),
            ("Planning ahead reduces stress and ensures preparation", "Spontaneity allows flexibility and serendipitous discovery"),
            ("Journaling processes thoughts through written reflection", "Meditation processes thoughts through quiet observation"),
            ("Podcasts provide information during other activities", "Audiobooks offer immersive narrative experiences"),
            ("Indoor plants bring nature inside living spaces", "Outdoor gardens connect with soil and seasons"),
            ("Detailed itineraries maximize travel experience efficiency", "Loose plans allow organic discovery and flexibility"),
            ("Frequent short trips provide regular refreshment breaks", "Longer annual trips allow deeper destination immersion"),
            ("Baths provide relaxing soaking contemplation time", "Showers offer efficient refreshing cleanliness quickly"),
            ("Breakfast as largest meal fuels active mornings", "Dinner as main meal enables social evening gatherings"),
            ("Scheduled social time ensures regular relationship maintenance", "Spontaneous socializing feels more natural and authentic"),
            ("Simple consistent wardrobe reduces daily decision fatigue", "Varied wardrobe enables expressive daily creativity"),
            ("Learning languages expands cultural understanding breadth", "Deepening native language improves precise expression"),
            ("Making bed starts day with small accomplished task", "Leaving bed unmade saves time for larger priorities"),
            ("Thank-you notes show thoughtful personal appreciation", "Verbal thanks provides immediate direct acknowledgment"),
            ("Collecting items preserves memories and curates interests", "Minimizing possessions reduces maintenance and clutter"),
        ]
    },
    "scientific_facts": {
        "description": "Scientific concepts with different framings",
        "pairs": [
            ("Gravity attracts objects based on their mass fundamentally", "Objects fall due to spacetime curvature geometry"),
            ("Evolution selects traits that enhance survival fitness", "Species change through random mutation and selection"),
            ("Light behaves as both particle and wave simultaneously", "Photons exhibit complementary quantum properties"),
            ("Entropy increases in isolated systems over time", "Disorder naturally increases without energy input"),
            ("Atoms contain protons neutrons and electrons fundamentally", "Matter consists of quarks and leptons ultimately"),
            ("Chemical bonds share or transfer electrons between atoms", "Molecular forces arise from electromagnetic interactions"),
            ("Stars fuse hydrogen into helium releasing energy", "Stellar nucleosynthesis creates heavier elements progressively"),
            ("Plate tectonics drives continental drift movements", "Mantle convection reshapes Earth's surface continuously"),
            ("Neurons transmit signals through electrochemical processes", "Brain activity emerges from neural network patterns"),
            ("Vaccines train immune systems to recognize pathogens", "Immunization prepares antibody responses to infections"),
            ("Antibiotics kill bacteria or inhibit their growth", "Antimicrobials target specific cellular mechanisms selectively"),
            ("Greenhouse gases trap heat in Earth's atmosphere", "Radiative forcing increases global temperature averages"),
            ("Black holes have gravity so strong light cannot escape", "Event horizons mark boundaries of no return"),
            ("DNA replication copies genetic information precisely", "Hereditary instructions pass through molecular mechanisms"),
            ("Photosynthesis converts light energy into chemical bonds", "Plants transform solar radiation into stored glucose"),
            ("Quantum entanglement links particles across distances", "Correlated states persist regardless of separation"),
            ("Superconductors conduct electricity without resistance", "Zero-resistance materials enable lossless current flow"),
            ("Tidal forces result from differential gravitational pull", "Moon's gravity creates ocean bulges on Earth"),
            ("Natural selection favors adaptive traits over generations", "Evolutionary pressure shapes species characteristics gradually"),
            ("Nuclear fission splits heavy atoms releasing energy", "Chain reactions multiply neutron-induced atomic divisions"),
            ("Climate describes long-term weather pattern averages", "Atmospheric conditions vary over extended timescales"),
            ("Genetic mutations introduce variation into populations", "DNA copying errors create hereditary diversity"),
            ("Electromagnetic waves propagate through oscillating fields", "Light travels as self-reinforcing field disturbances"),
            ("Mitosis divides cells for growth and repair", "Cell division duplicates genetic material precisely"),
            ("Ocean currents distribute heat around the globe", "Thermohaline circulation regulates planetary temperatures"),
            ("Immune responses identify and eliminate foreign threats", "Defense mechanisms distinguish self from non-self"),
            ("Catalysts speed reactions without being consumed", "Enzymes lower activation energy for biochemical processes"),
            ("Magnetic fields arise from moving electric charges", "Current flow generates perpendicular force fields"),
            ("Ecosystems cycle nutrients through interconnected organisms", "Biogeochemical processes recycle essential elements continuously"),
            ("Atmospheric pressure decreases with increasing altitude", "Air density reduces at higher elevations naturally"),
        ]
    },
    "business": {
        "description": "Organizational strategy approaches",
        "pairs": [
            ("Centralized teams enable consistent decision-making across organization", "Distributed teams allow faster local adaptation to conditions"),
            ("Hierarchical structures provide clear accountability chains", "Flat organizations enable rapid information flow everywhere"),
            ("Top-down strategy ensures aligned organizational direction", "Bottom-up innovation captures frontline insights effectively"),
            ("Standardized processes ensure consistent quality outcomes", "Flexible processes adapt to unique situation requirements"),
            ("Dedicated specialists develop deep domain expertise", "Cross-functional generalists see broader system connections"),
            ("Long-term planning provides strategic direction stability", "Iterative adaptation responds to market changes quickly"),
            ("Large established vendors offer stability and support", "Agile startups provide innovative cutting-edge solutions"),
            ("In-house development maintains control over core capabilities", "Outsourcing enables focus on core competencies"),
            ("Formal meetings ensure documented decisions and accountability", "Informal communication enables faster coordination"),
            ("Comprehensive documentation preserves institutional knowledge", "Tacit knowledge transfer through mentorship and practice"),
            ("Quarterly targets provide measurable progress milestones", "Continuous flow optimizes for sustainable throughput"),
            ("Risk-averse approaches protect existing valuable assets", "Risk-taking approaches capture new opportunities boldly"),
            ("Process optimization maximizes efficiency of operations", "Outcome focus allows flexible method selection"),
            ("Detailed budgets control spending precisely predictably", "Flexible funding responds to emerging opportunities"),
            ("Experienced leadership provides proven judgment wisdom", "Fresh perspectives challenge established assumptions"),
            ("Established markets offer predictable revenue streams", "Emerging markets provide growth potential opportunities"),
            ("Vertical integration controls entire value chain", "Strategic partnerships leverage external capabilities"),
            ("Proprietary technology creates competitive moats defensibly", "Open standards enable ecosystem collaboration growth"),
            ("Premium positioning commands higher profit margins", "Volume strategy captures market share broadly"),
            ("Conservative growth preserves culture and quality", "Aggressive expansion captures market opportunities"),
            ("Physical presence enables direct customer relationships", "Digital channels provide scalable reach efficiently"),
            ("Specialized focus builds distinctive category leadership", "Diversification spreads risk across multiple markets"),
            ("Internal promotion develops loyal institutional knowledge", "External hiring brings fresh perspectives quickly"),
            ("Comprehensive training ensures consistent capability baseline", "Learning by doing develops practical skills faster"),
            ("Centralized procurement achieves volume discounts", "Local sourcing enables faster supplier responsiveness"),
            ("Formal performance reviews provide structured feedback", "Continuous feedback enables timely course corrections"),
            ("Job security enables long-term thinking and investment", "Performance pressure drives continuous improvement"),
            ("Competitive compensation attracts top talent effectively", "Mission alignment attracts intrinsically motivated people"),
            ("Office presence enables spontaneous collaboration naturally", "Remote work enables deep focused productivity"),
            ("Consensus building ensures broad buy-in and support", "Decisive action enables rapid execution speed"),
            ("Data-driven decisions reduce bias and increase objectivity", "Intuitive decisions leverage tacit pattern recognition"),
            ("Formal authority enables clear decision rights", "Influence-based leadership builds genuine commitment"),
            ("Structured onboarding ensures consistent new hire experience", "Immersive onboarding accelerates productive contribution"),
            ("Annual reviews provide comprehensive performance assessment", "Real-time feedback enables immediate improvement"),
            ("Specialized roles enable deep expertise development", "Rotating roles build broad organizational understanding"),
            ("Formal career paths provide clear advancement progression", "Organic growth allows pursuit of emerging opportunities"),
            ("Centralized hiring maintains consistent culture standards", "Team-level hiring enables best local fit assessment"),
            ("Comprehensive benefits packages attract diverse talent", "Flexible benefits allow personalized compensation mix"),
            ("Structured mentorship programs ensure knowledge transfer", "Organic mentorship relationships form naturally authentically"),
            ("Formal recognition programs celebrate achievements visibly", "Informal recognition provides immediate authentic appreciation"),
            ("Cross-functional committees coordinate complex initiatives", "Autonomous teams own end-to-end delivery independently"),
            ("Executive steering provides strategic direction alignment", "Self-organizing teams adapt to local conditions"),
            ("Standardized tools ensure consistent collaboration platform", "Tool flexibility allows best-fit selection locally"),
            ("Centralized analytics provides organization-wide insights", "Embedded analytics enables team-specific customization"),
            ("Formal change management ensures smooth transitions", "Organic adoption allows natural evolution pace"),
            ("Comprehensive policies ensure consistent fair treatment", "Principle-based guidance enables contextual judgment"),
            ("Structured meetings ensure productive focused time use", "Unstructured time enables creative exploration"),
            ("Formal escalation paths ensure issue resolution clarity", "Direct resolution empowers frontline problem solving"),
            ("Centralized communications ensure consistent messaging", "Distributed voices provide authentic diverse perspectives"),
            ("Strategic planning cycles ensure coordinated investments", "Continuous planning enables responsive adaptation"),
        ]
    },
    "tech_philosophy": {
        "description": "Software Holy Wars",
        "pairs": [
            ("Tabs provide consistent alignment across all editors", "Spaces ensure identical display in any environment"),
            ("Vim offers powerful modal editing for efficiency", "Emacs provides extensible environment for customization"),
            ("Static typing catches errors at compile time", "Dynamic typing enables rapid prototyping flexibility"),
            ("Monolithic architectures simplify deployment and debugging", "Microservices enable independent scaling and deployment"),
            ("SQL databases ensure ACID compliance and data integrity", "NoSQL databases provide flexible schema and horizontal scaling"),
            ("Object-oriented programming models real-world entities naturally", "Functional programming ensures predictable stateless behavior"),
            ("REST APIs use standard HTTP methods for simplicity", "GraphQL allows clients to request exactly needed data"),
            ("Agile methodology adapts quickly to changing requirements", "Waterfall methodology provides clear milestones and documentation"),
            ("Test-driven development ensures code correctness from start", "Behavior-driven development aligns tests with business requirements"),
            ("Compiled languages offer maximum runtime performance", "Interpreted languages provide faster development cycles"),
            ("Monorepos simplify dependency management across projects", "Polyrepos allow independent versioning and deployment"),
            ("Inheritance enables code reuse through class hierarchies", "Composition provides flexible behavior through object assembly"),
            ("Mutable state enables efficient in-place modifications", "Immutable data structures prevent unexpected side effects"),
            ("Synchronous code is easier to reason about and debug", "Asynchronous code better utilizes system resources"),
            ("Centralized version control simplifies access control", "Distributed version control enables offline work and branching"),
            ("Relational databases normalize data to prevent redundancy", "Document databases denormalize for read performance"),
            ("Server-side rendering improves initial page load and SEO", "Client-side rendering provides richer interactive experiences"),
            ("Convention over configuration reduces boilerplate decisions", "Explicit configuration provides maximum flexibility control"),
            ("Early optimization prevents performance problems proactively", "Premature optimization wastes effort on non-bottlenecks"),
            ("DRY principle eliminates redundant code maintenance burden", "Some duplication is better than wrong abstraction"),
            ("Strict linting enforces consistent code style automatically", "Flexible linting allows developer judgment and preferences"),
            ("Long functions keep related logic together visibly", "Short functions improve readability and testability"),
            ("Comments explain the why behind code decisions", "Self-documenting code eliminates outdated comment drift"),
            ("Defensive programming handles all possible edge cases", "Fail-fast approach surfaces problems immediately clearly"),
            ("Design patterns provide proven solutions to problems", "Simple code is better than pattern-heavy abstractions"),
            ("Framework adoption accelerates development with conventions", "Library composition provides maximum flexibility control"),
            ("Trunk-based development enables continuous integration", "Feature branches isolate work in progress safely"),
            ("Code reviews catch bugs and share knowledge across team", "Pair programming provides immediate feedback and collaboration"),
            ("Comprehensive logging aids debugging and monitoring", "Minimal logging reduces noise and storage costs"),
            ("Strong typing prevents entire categories of bugs", "Weak typing reduces ceremony and boilerplate code"),
            ("Vertical scaling is simpler to implement and manage", "Horizontal scaling provides better fault tolerance"),
            ("Caching improves performance for repeated operations", "Cache invalidation complexity often outweighs benefits"),
            ("Dependency injection improves testability and flexibility", "Direct instantiation is simpler and more explicit"),
            ("Service mesh handles cross-cutting networking concerns", "Library approach keeps networking logic explicit"),
            ("Event sourcing provides complete audit trail history", "Current state storage is simpler to query"),
            ("Kubernetes orchestration handles complex deployments well", "Simpler deployment tools reduce operational complexity"),
            ("Infrastructure as code ensures reproducible environments", "Manual configuration allows quick one-off changes"),
            ("Container images ensure consistent deployment environments", "Virtual machines provide stronger isolation guarantees"),
            ("Message queues decouple services for resilience", "Direct calls are simpler and easier to debug"),
            ("Feature flags enable gradual rollouts safely", "Simple deployments avoid flag complexity accumulation"),
            ("OAuth provides delegated authorization for third-party access", "JWT provides self-contained tokens for stateless authentication"),
            ("gRPC provides efficient binary protocol communication", "REST provides simple human-readable API design"),
            ("WebSockets enable real-time bidirectional communication", "Server-sent events suffice for server-to-client streaming"),
            ("Serverless functions scale automatically with demand", "Dedicated servers provide predictable performance costs"),
            ("GraphQL subscriptions handle real-time updates elegantly", "Polling is simpler and works everywhere reliably"),
            ("Kafka handles high-throughput event streaming reliably", "RabbitMQ provides flexible routing for messages"),
            ("PostgreSQL provides robust relational database features", "MySQL offers widespread support and familiarity"),
            ("Redis excels at caching and session storage", "Memcached provides simple distributed caching"),
            ("Elasticsearch enables powerful full-text search capabilities", "PostgreSQL full-text search avoids additional infrastructure"),
            ("Terraform manages infrastructure declaratively across clouds", "CloudFormation integrates deeply with AWS services"),
        ]
    }
}

total_pairs = sum(len(cat["pairs"]) for cat in DATASET.values())
print(f"✅ Dataset loaded: {total_pairs} pairs across {len(DATASET)} categories")

In [ ]:
# Cell 6: Core Functions

def get_embedding(text: str, model, tokenizer) -> np.ndarray:
    """Extract embedding via mean pooling over last hidden layer."""
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]
        embedding = hidden_states[0, 1:, :].mean(dim=0)
    return embedding.cpu().numpy().astype(np.float32)


def get_nll(text: str, model, tokenizer) -> Tuple[float, float, int]:
    """
    Calculate NLL (Negative Log Likelihood) for text.
    Using NLL instead of PPL because it's linear (better for correlation).
    Returns: (nll, perplexity, n_tokens)
    """
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    n_tokens = inputs["input_ids"].shape[1]
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        nll = outputs.loss.item()
        ppl = torch.exp(outputs.loss).item()
    return nll, ppl, n_tokens


def compute_preference(nll_a: float, nll_b: float) -> float:
    """Preference = NLL(B) - NLL(A). Positive = prefers A."""
    return nll_b - nll_a


def uniformity_score(embeddings: np.ndarray) -> float:
    """Average pairwise cosine similarity."""
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized = embeddings / (norms + 1e-10)
    kernel = normalized @ normalized.T
    n = kernel.shape[0]
    idx = np.triu_indices(n, k=1)
    return float(np.mean(kernel[idx]))


def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10))


def bootstrap_ci(data: np.ndarray, n_bootstrap: int = N_BOOTSTRAP) -> Tuple[float, float]:
    """Bootstrap 95% CI for mean."""
    np.random.seed(RANDOM_SEED)
    means = [np.mean(np.random.choice(data, len(data), replace=True)) for _ in range(n_bootstrap)]
    return (float(np.percentile(means, 2.5)), float(np.percentile(means, 97.5)))


def bootstrap_correlation_ci(x: np.ndarray, y: np.ndarray, n_bootstrap: int = N_BOOTSTRAP) -> Tuple[float, float]:
    """Bootstrap 95% CI for Pearson correlation."""
    np.random.seed(RANDOM_SEED)
    n = len(x)
    corrs = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, n, replace=True)
        r, _ = stats.pearsonr(x[idx], y[idx])
        corrs.append(r)
    return (float(np.percentile(corrs, 2.5)), float(np.percentile(corrs, 97.5)))


print("✅ Functions defined")

In [ ]:
# Cell 7: Load Model
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"Loading {config['display']}...")
print(f"This may take 2-5 minutes...")

tokenizer = AutoTokenizer.from_pretrained(
    config["hf_name"],
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    config["hf_name"],
    torch_dtype=config["dtype"],
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ Model loaded on: {model.device}")

In [ ]:
# Cell 8: Run Experiment
print(f"\n{'='*60}")
print(f"RUNNING OUTPUT-CORRELATION EXPERIMENT")
print(f"Model: {config['display']}")
print(f"{'='*60}")

results = {}
all_pair_similarities = []
all_pair_preferences = []
all_pair_categories = []

processed = 0

for category_name, category_data in DATASET.items():
    print(f"\n--- {category_name.upper()} ---")

    pairs = category_data["pairs"]
    embeddings_a, embeddings_b = [], []
    pair_similarities, pair_preferences = [], []
    nll_data = []

    for stmt_a, stmt_b in pairs:
        processed += 1
        print(f"  [{processed:03d}/{total_pairs}] Processing...", end="\r")

        # Embeddings
        emb_a = get_embedding(stmt_a, model, tokenizer)
        emb_b = get_embedding(stmt_b, model, tokenizer)
        embeddings_a.append(emb_a)
        embeddings_b.append(emb_b)

        sim = cosine_similarity(emb_a, emb_b)
        pair_similarities.append(sim)

        # NLL
        nll_a, ppl_a, n_a = get_nll(stmt_a, model, tokenizer)
        nll_b, ppl_b, n_b = get_nll(stmt_b, model, tokenizer)
        pref = compute_preference(nll_a, nll_b)
        pair_preferences.append(pref)

        nll_data.append({"nll_a": nll_a, "nll_b": nll_b, "ppl_a": ppl_a, "ppl_b": ppl_b})

        all_pair_similarities.append(sim)
        all_pair_preferences.append(pref)
        all_pair_categories.append(category_name)

    # Category stats
    embeddings_a = np.array(embeddings_a)
    embeddings_b = np.array(embeddings_b)
    pair_preferences = np.array(pair_preferences)

    u_a = uniformity_score(embeddings_a)
    u_b = uniformity_score(embeddings_b)
    ua = u_a - u_b

    results[category_name] = {
        "n_pairs": len(pairs),
        "ua_score": ua,
        "preference_nll_mean": float(np.mean(pair_preferences)),
        "preference_nll_std": float(np.std(pair_preferences)),
        "pct_prefer_a": float(np.mean(pair_preferences > PREF_THRESHOLD) * 100),
        "pct_prefer_b": float(np.mean(pair_preferences < -PREF_THRESHOLD) * 100),
        "mean_nll_a": float(np.mean([d["nll_a"] for d in nll_data])),
        "mean_nll_b": float(np.mean([d["nll_b"] for d in nll_data])),
    }

    print(f"  UA: {ua:+.4f} | Pref: {np.mean(pair_preferences):+.4f} | "
          f"%A: {results[category_name]['pct_prefer_a']:.1f}%")

print(f"\n✅ Processed {processed} pairs")

In [ ]:
# Cell 9: Correlation Analysis
print("\n" + "="*60)
print("CORRELATION ANALYSIS")
print("="*60)

# Category-level correlation
categories = list(results.keys())
ua_values = np.array([results[k]["ua_score"] for k in categories])
pref_values = np.array([results[k]["preference_nll_mean"] for k in categories])

cat_corr, cat_p = stats.pearsonr(ua_values, pref_values)
cat_ci = bootstrap_correlation_ci(ua_values, pref_values)

print(f"\nCategory-Level (n={len(categories)}):")
print(f"  Pearson r: {cat_corr:.4f}")
print(f"  p-value:   {cat_p:.4f}")
print(f"  95% CI:    [{cat_ci[0]:.4f}, {cat_ci[1]:.4f}]")

# Per-pair correlation
all_sim = np.array(all_pair_similarities)
all_pref = np.array(all_pair_preferences)

pair_corr, pair_p = stats.pearsonr(all_sim, all_pref)
pair_ci = bootstrap_correlation_ci(all_sim, all_pref)

print(f"\nPer-Pair (n={len(all_sim)}):")
print(f"  Pearson r: {pair_corr:.4f}")
print(f"  p-value:   {pair_p:.6f}")
print(f"  95% CI:    [{pair_ci[0]:.4f}, {pair_ci[1]:.4f}]")

# Interpretation
print("\n" + "-"*60)
if cat_p < 0.05 and abs(cat_corr) > 0.5:
    print("🎯 STRONG FINDING: UA correlates with output preference!")
elif cat_p < 0.05:
    print("📊 MODERATE: Some correlation detected")
else:
    print("⚖️ NEUTRAL: No significant category-level correlation")

# Store in results
results["_correlation"] = {
    "category": {"r": cat_corr, "p": cat_p, "ci": cat_ci},
    "pair": {"r": pair_corr, "p": pair_p, "ci": pair_ci}
}

In [ ]:
# Cell 10: Summary Table
print("\n" + "="*80)
print(f"SUMMARY: {config['display']}")
print("="*80)
print(f"{'Category':<25} {'UA':>8} {'Pref(NLL)':>12} {'%A':>8} {'%B':>8}")
print("-"*65)

for cat, data in results.items():
    if cat.startswith("_"):
        continue
    print(f"{cat:<25} {data['ua_score']:>+8.4f} {data['preference_nll_mean']:>+12.5f} "
          f"{data['pct_prefer_a']:>7.1f}% {data['pct_prefer_b']:>7.1f}%")

print("-"*65)
print(f"\nCorrelation (UA vs Pref): r = {cat_corr:.4f}, p = {cat_p:.4f}")

In [ ]:
# Cell 11: Save Results
results["_metadata"] = {
    "model": MODEL_NAME,
    "model_display": config["display"],
    "timestamp": datetime.now().isoformat(),
    "n_bootstrap": N_BOOTSTRAP,
    "experiment": "output_correlation",
    "version": "1.0.0"
}

output_file = f"output_correlation_{MODEL_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✅ Results saved to: {output_file}")
print("\nDownload this file for your records!")

# Also display download link in Colab
from google.colab import files
files.download(output_file)

---

## Next Steps

1. Run this notebook for each model:
   - Change `MODEL_NAME` in Cell 2
   - Run all cells again

2. Models to test:
   - `llama` (baseline - should show neutral)
   - `mistral` (baseline - should show neutral)
   - `gemma` (slight asymmetry expected)
   - `apertus` (strong asymmetry expected on scientific)

3. Compare results across models

---

*Output-Correlation Experiment v1.0*
